In [1]:
from selenium import webdriver
import time
from openpyxl import Workbook
import datetime
from selenium.webdriver.common.keys import Keys
import pyperclip
import pandas as pd


k=0
list=['나이키 에어맥스 97', '아디다스 오즈위고', '반스 올드스쿨', '호킨스 푸파', '리복 로얄 하이페리움 2', '누오보 바베트', '컨버스 척테일러 올스타', '뉴발란스 문빔', '버켄스탁 아리조나', '퓨마 스웨이드 빈티지']
fac_price=[199000,129000,69000,39000,69000,59000,69000,108000,95000,119000]
for i in list:

    # datetime
    now = datetime.datetime.now()
    today = now.strftime('%Y-%m-%d')

    # 엑셀
    wb = Workbook(write_only=True)
    ws = wb.create_sheet(today)
    ws.append(['작성날짜', '제목', '모델', '가격'])

    # 중고나라 들어가기
    driver = webdriver.Chrome("./chromedriver.exe")
    driver.implicitly_wait(3)
    driver.get('https://cafe.naver.com/joonggonara')
    driver.maximize_window()
    time.sleep(1)

    # 로그인 버튼을 찾고 클릭합니다.
    login_btn = driver.find_element_by_css_selector('#gnb_login_button')
    login_btn.click()
    time.sleep(1)

    # id, pw 입력할 곳을 찾습니다.
    tag_id = driver.find_element_by_name('id')
    tag_pw = driver.find_element_by_name('pw')
    tag_id.clear()
    time.sleep(1)

    # id 입력
    tag_id.click()
    pyperclip.copy('id')
    tag_id.send_keys(Keys.CONTROL, 'v')
    time.sleep(1)

    # pw 입력
    tag_pw.click()
    pyperclip.copy('pw')
    tag_pw.send_keys(Keys.CONTROL, 'v')
    time.sleep(1)

    # 로그인 버튼을 클릭합니다
    login_btn = driver.find_element_by_id('log.login')
    login_btn.click()

    # 검색할 물건
    thing = i

    # 총 몇 페이지 자료를 모을지 선택
    total_page = 10

    # 페이지 개수 나누기
    total_next = total_page // 10
    last_page = total_page - total_next * 10

    # 검색
    driver.find_element_by_css_selector('#topLayerQueryInput').send_keys(thing)
    driver.find_element_by_css_selector('#cafe-search .btn').click()
    time.sleep(1)

    # iframe 들어가기
    driver.switch_to.frame('cafe_main')

    # 제목만으로 바꾸기
    driver.find_element_by_css_selector('#currentSearchByTop').click()
    time.sleep(1)
    driver.find_elements_by_css_selector('#sl_general li')[1].click()
    time.sleep(1)
    driver.find_element_by_css_selector('.btn-search-green').click()
    time.sleep(1)

    # 크롤링 함수 정의(만약 "이전" 버튼이 있으면 num은 1 아니면 0)
    def crolling(num):
        # 게시글 들어가는 반복문
        with_before = 0
        for i in range(len(driver.find_elements_by_css_selector('.article'))):

            # 게시글 들어가기
            articles = driver.find_elements_by_css_selector('a.article')[i]
            articles.click()
            time.sleep(1)

            # 정보추출
            write_date = driver.find_element_by_css_selector('.date').text
            product_title = driver.find_element_by_css_selector('h3.title_text').text
            url = driver.find_element_by_css_selector('.button_url').get_attribute('href')
            # 가격을 못찾으면 그냥 빈칸 입력
            try:
                # 가격 문자열을 숫자로 바꾸기
                product_price_str = driver.find_element_by_css_selector('.ProductPrice').text
                price_no_won = product_price_str[:-1]
                price_no_won_shim = price_no_won.replace(',', '')
                product_price = int(price_no_won_shim)
            except:
                # 제목에서 가격 문자열 추출
                try:
                    product_title = product_title.replace('[', '')
                    product_title = product_title.replace(']', '&')
                    product_price_str = product_title.split('&')[-2]

                    # 가격 문자열을 숫자로 바꾸기
                    price_no_won = product_price_str[:-1]
                    price_no_won_shim = price_no_won.replace(',', '')
                    product_price = int(price_no_won_shim)
                except:
                    product_price = ''

            # 엑셀에 작성
            ws.append([write_date, product_title, i, product_price,fac_price[k]])

            # 뒤로가기
            driver.back()
            driver.switch_to.frame('cafe_main')

        # 다음 게시글 page 이동
        pages = driver.find_elements_by_css_selector('.prev-next a')[page + 1 + num]
        pages.click()

    for i in range(total_next):
        # 마지막 10단위 페이지일 때
        if i == 0:
            # 다음페이지 클릭 반복문
            for page in range(10):
                crolling(0)
        elif i > 0 and i != total_next-1:
            for page in range(10):
                crolling(1)
        elif i == total_next-1:
            for page in range(last_page):
                crolling(1)


    # selenium 끝내고 엑셀 파일 저장
    driver.quit()
    wb.save(f'중고나라 {today}{thing} 매물.xlsx')

    k+=1
    # 데이터프레임 생성
    df = pd.read_excel(f'C:\\Users\\USER\201910809\\기계학습\\중고나라 {today}{thing} 매물.xlsx')
    # 가격이 비이상적인 데이터 삭제하기
    q1 = df['가격'].quantile(0.25)
    q3 = df['가격'].quantile(0.75)
    iqr = q3 - q1
    condition = (df['가격'] > q3 + 1.5 * iqr) | (df['가격'] < q1 - 1.5 * iqr)
    df.drop(df[condition].index, inplace=True)

print(df.describe())

C:\Users\USER\AppData\Local\Temp/ipykernel_2956/3255910402.py:23: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome("./chromedriver.exe")
C:\Users\USER\AppData\Local\Temp/ipykernel_2956/3255910402.py:30: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  login_btn = driver.find_element_by_css_selector('#gnb_login_button')


KeyboardInterrupt: 